# Use Retrieval-Augmented Generation

## Choose Models

This section uses LangChain to specify which Ollama model to query and which embeddings model to use.

In [202]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains.combine_documents import create_stuff_documents_chain

#llm = Ollama(model="llama2")
llm = Ollama(model="mistral")

## Load Information About SLUUG

This section of code will read in information about the St. Louis Unix Users Group. That information is put into a LangChain PromptTemplate.

In [203]:
system_prompt_file = "system_prompt.md"
with open(system_prompt_file, encoding="utf-8") as f:
    system_prompt = f.read()

prompt = PromptTemplate.from_template(
    system_prompt + 
    """
    {context}

    Question: {question}

    Answer:""")

## Load Information about the February-July 2021 Presentations

This section of code will read in information about the BASE and MAIN presentations that were given on February-July 2021 and store it in a variable.

In [204]:
markdown_files = ["2021-02-10-SLUUG.md", "2021-03-10-SLUUG.md", "2021-04-14-SLUUG.md", "2021-05-12-SLUUG.md", "2021-06-09-SLUUG.md", "2021-07-14-SLUUG.md"]
doc_str = ""
for markdown_file in markdown_files:
    with open(markdown_file, encoding="utf-8") as f:
        doc_str += f.read()

Next, we need to split the large `doc_str` into multiple "chunks." This code will use the MarkdownHeaderTextSplitter from LangChain to split the `doc_str` on the Markdown Headers. Each second-level header will be put in its own chunk. This means each Presentation will have it's own chunk.

In [205]:
headers_to_split_on = [
    ("#", "Meeting Title"),
    ("##", "Presenataion Type"),
]
text_splitter = MarkdownHeaderTextSplitter(headers_to_split_on)
documents = text_splitter.split_text(doc_str)

for chunk in documents:
    print(chunk)
    print("\n")

page_content="The BASE presentation on February 10th, 2021 was titled: Speedbar.  \nThe presenter was Steve Stegman.  \nThe presentation abstract was as follows:\nEmacs' Speedbar for fast file access.  \nSpeedBar is a tool (written in EMACS) for organizing or listing files and directories  \nSpeedbar is an integral part of Emacs which has been in use since version 23.2 in 2010. Entering “speedbar” in the emacs command line causes another frame to be opened to graphically show the current directory tree which can be traversed upwards and downwards from the current position. It shows all of the files and directories and each can be examined with a single mouse click. If the file is not text but a jpg or pdf or some other file type, an auxiliary program will be called to display the file in an emacs buffer." metadata={'Meeting Title': 'St. Louis UNIX Users Group (SLUUG) - 2021-02-10', 'Presenataion Type': 'BASE Presentation'}


page_content="The MAIN presentation on February 10th, 2021 wa

## Create Embeddings Of Each Chunk

This section will try to figure out the symantic meaning of each chunk. To do this it will use the embeddings model specified above. 

Next, it will then store the embeddings in the [FAISS](https://faiss.ai/) vectorstore. From the vectorstore, a retriever is created. The `search_kwargs` tells the retriever how many chunks to return.

Lastly, it will create a LangChain.

In [213]:
embeddings = OllamaEmbeddings(model="nomic-embed-text")
vectorstore = FAISS.from_documents(documents, embeddings)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 12})
chain = create_stuff_documents_chain(llm, prompt)

## Augmented Prompts

This section will send information about SLUUG and Information about the BASE and MAIN presentations that were given on February-July 2021 along with the Question.

First, ask the LLM to identify the BASE presentation.

In [214]:
retrieved = retriever.invoke("What was the BASE presentation given to the St. Louis Unix Users Group on February 2021?")
print(f"Retrieved {len(retrieved)} chunks.")
for chunk in retrieved:
    print(chunk)
    print("\n")
print(f"Answer: {chain.invoke({"context": retrieved, "question": "What was the BASE presentation given to the St. Louis Unix Users Group on February 2021?"})}")

Retrieved 12 chunks.
page_content='The MAIN presentation on April 14th, 2021 was titled: COSI (Common Operating System Interface)  \nThe presenter was  Steven Borrelli.  \nThe presentation abstract was as follows:\nThis talk introduces COSI, the Common Operating System Interface, which defines an API for the configuration of container operating systems.  \nWhile there has been innovation in the development of minimal Linux distributions for running Kubernetes, the distributions diverge from each other in terms of management and API endpoints. Similar to other projects such as the Container Network Interface (CNI) , COSI has a focus on the configuration of the underlying (Linux) OS, providing Protocol Buffer definitions and a gRPC API reference implementation for configuration settings such as DNS, network, and kernel settings. By utilizing a flexible plugin system, multiple backend implementations can provide a consistent API to consumers.' metadata={'Meeting Title': 'St. Louis UNIX Us

In [208]:
retrieved = retriever.invoke("When was a presentation about Emacs Speedbar given and by who?")
print(f"Retrieved {len(retrieved)} chunks.")
for chunk in retrieved:
    print(chunk)
    print("\n")
print(f"Answer: {chain.invoke({"context": retrieved, "question": "When was a presentation about Emacs Speedbar given and by who?"})}")

Retrieved 3 chunks.
page_content="The BASE presentation on February 10th, 2021 was titled: Speedbar.  \nThe presenter was Steve Stegman.  \nThe presentation abstract was as follows:\nEmacs' Speedbar for fast file access.  \nSpeedBar is a tool (written in EMACS) for organizing or listing files and directories  \nSpeedbar is an integral part of Emacs which has been in use since version 23.2 in 2010. Entering “speedbar” in the emacs command line causes another frame to be opened to graphically show the current directory tree which can be traversed upwards and downwards from the current position. It shows all of the files and directories and each can be examined with a single mouse click. If the file is not text but a jpg or pdf or some other file type, an auxiliary program will be called to display the file in an emacs buffer." metadata={'Meeting Title': 'St. Louis UNIX Users Group (SLUUG) - 2021-02-10', 'Presenataion Type': 'BASE Presentation'}


page_content='The MAIN presentation on Ju

In [209]:
retrieved = retriever.invoke("Which presentations were related to source code versioning?")
print(f"Retrieved {len(retrieved)} chunks.")
for chunk in retrieved:
    print(chunk)
    print("\n")
print(f"Answer: {chain.invoke({"context": retrieved, "question": "Which presentations were related to source code versioning?"})}")

Retrieved 3 chunks.
page_content='The MAIN presentation on June 9th, 2021 was titled: git: peering behind the curtain  \nThe presenter was Robert Citek.  \nThe presentation abstract was as follows:\nAfter Robert\'s "A Gentle Introduction To git" last February, this will be a deeper dive, getting into some of the "behind the scenes" stuff that git does: where it stores files, how it stores metadata about the files, and how the files are organized.  \nIn a previous talk, we went over the basics of using git as a version control system, covering the working folder, the staging area, and the repository. In this talk, we will go behind the scenes to see how git works its magic. We will cover the abstractions git uses, in particular, a key-value store, hashes, and graphs. We will then dig into how git implements and uses those abstractions to create a version control system using files and the file system. We will walk through examples of using the back-end commands to build a commit history